In [88]:
import pandas as pd
import re
import os
import warnings
warnings.filterwarnings('ignore')

#pd.set_option('display.max_rows', 500)

### Zemberek

In [89]:
import jpype

In [90]:
libjvmpath = "C:\Program Files\Java\jre1.8.0_341\\bin\server\jvm.dll"
jar = r"C:\Users\yucel\Desktop\staj\JupiterLab\zemberek\zemberek-full.jar"

In [91]:
if not jpype.isJVMStarted():
    jpype.startJVM(jvmpath=libjvmpath, classpath=jar)

In [92]:
TurkishMorphology = jpype.JClass('zemberek.morphology.TurkishMorphology')
morphology = TurkishMorphology.createWithDefaults()

In [93]:
LanguageIdentifier = jpype.JClass('zemberek.langid.LanguageIdentifier')
lid = LanguageIdentifier.fromInternalModels();

# STEMMING / STEP 7

Kelimeleri köke indirgeme işlemi

## READ FILE

In [94]:
sentences = pd.read_csv("yazilim_uzmani_sentences_v2.csv")
sentences = sentences.drop(["Unnamed: 0"],axis=1)
sentences.reset_index(inplace=True)
sentences = sentences.drop(["index"],axis=1)
stem_sentences=sentences.copy()
stem_sentences.head()

,PostId,Sentence,Language
0,Post0,"dünyada ülkede, farklı dil yaklaşık",Türkçe
1,Post0,firma kullanılan erp yazılımı SPCL38SPCL busin...,Türkçe
2,Post0,üniversitelerin lisans bölümlerinden mezun,Türkçe
3,Post0,erp pojelerinde test uzmanı olarak yıl tecrübeli,Türkçe
4,Post0,analitik matematiksel düşünme yönü kuvvetli,Türkçe


## STEMMING FUNCTIONS

### Turkish

In [95]:
def stemming_turkish(text):
    deneme=""
    stem_text=""
    analysis: jpype.java.util.ArrayList = (
    morphology.analyzeAndDisambiguate(text).bestAnalysis()
    )
    for i in analysis:
        #print(i)
        if "UNK" in  str(i.getLemmas()[0]):
            stem_text=stem_text+" "+str(i.getStem())
            deneme = deneme + " "+str(i.getLemmas()[0])+str(i.getEnding())
        elif "Verb" in  str(i) or "Punc" in  str(i) :
            stem_text=stem_text+" "+str(i.getLemmas()[0])
        else:
            stem_text=stem_text+" "+str(i.getLemmas()[0])
            deneme = deneme + " "+str(i.getLemmas()[0])+str(i.getEnding())
    return stem_text

In [96]:
stemming_turkish("lise mezunuysanız")

' lise mezun'

In [97]:
stem_sentences["Sentence"]=stem_sentences["Sentence"].apply(lambda text: stemming_turkish(str(text)))      

In [98]:
#stem_sentences["Sentence"]=stem_sentences["Sentence"].replace(to_replace=[","],value=" ",regex=True,inplace=False)
#stem_sentences["Sentence"] = stem_sentences["Sentence"].apply(lambda text: text.strip())

In [99]:
stem_sentences["Sentence"].head(60)

0                        dünya ülke , fark dil yaklaşık
1      firma kullan erp yazılım SPCL38SPCL business ...
2                         üniversite lisans bölüm mezun
3             erp pojelerinde test uzman ol yıl tecrübe
4                   analitik matematik düşün yön kuvvet
5                                      detay odak çalış
6                                           raporla yap
7                                       ekip çalış uyum
8                             problem çöz yetenek sahip
9                        i̇ş teslim tarih uyan i̇ş tani
10     program gereksinim karşıla emin , yeni yazılı...
11     bulun hata tür sorun çöz at adım özetle rapor...
12     hata potansiyel tanımla , risk değerlendir ye...
13                    yazılım test metodojilerine hakim
14                                   genel ni̇teli̇kler
15     üniversite , bilgisayar mühendis , yazılım mü...
16     yıl yazılım geliş deneyim yazılım geliş deney...
17     SPCL70SPCL , SPCL43SPCL , SPCL45SPCL core

# CLEANING MEANINGFUL EXPRESSIONS / STEP 8

In [100]:
meaningless_frequent_sentences_raw = pd.read_fwf(os.path.abspath('..')+"/Data/eliminated_frequent_sentence.txt")
meaningless_frequent_sentences= meaningless_frequent_sentences_raw["Sentences"].apply(lambda text: text.strip())
meaningless_frequent_sentences = meaningless_frequent_sentences.tolist()

In [101]:
meaningless_frequent_sentences

['i̇ş tani',
 'go',
 '"',
 'ş',
 'genel ni̇teli̇kler',
 'genel nitelik',
 'deneyelimli',
 'i̇ş tan',
 'ara nitelik',
 'a',
 'bölüm mezun',
 'b',
 'genel ni̇teli̇kler i̇ş tani',
 'tic',
 'yıl deneyim',
 'kelime ifade et ses kal güç var ruh',
 'muhteşem akustik yap değer zaman sahip çık',
 'i̇şte ruh kendi hayran bırak',
 'yıl tecrübe',
 'yaş aş',
 'i̇stanbul avrupa yaka ikamet et',
 'ara ni̇teli̇kler',
 'görev tanım',
 'mi',
 's',
 'i̇şi̇n tani',
 'alan yıl deneyim',
 'benzer pozisyon yıl deneyim',
 'i̇stanbul anadol yaka ikamet et',
 'tarih no lisans sahip özel istihdam büro',
 'alan yıl tecrübe',
 'i̇şin tan',
 'm',
 'zaman haydi cesaret , iş duy heyecan , coşku büyük ritm parça ol , yeni yol keşfet , yolcu biz birlikte ritim tut , ezber boz ritim güç güç kat mı perakende hareket yapı ilgi',
 'biz penti fark tını güç al ritm birlikte ayak uy',
 'i̇stenen yetenek uzman',
 'yıl deneyim',
 'ltd',
 'yıl tecrübe',
 'şti',
 'i̇stanbul avrupa yaka ikamet et',
 'bölüm mezun',
 'ara özellik',


In [102]:
def clean_frequent_expression(text):
    for exp in meaningless_frequent_sentences:
        text=text.replace(" "+exp+" "," ")
    return  text  

In [103]:
stem_sentences["Sentence"][46]

' üniversite bilgisayar , elektronik , matematik mühendis bölüm mezun ,'

In [104]:
stem_sentences["Sentence"]=stem_sentences["Sentence"].apply(lambda text: " "+text+" ")

In [105]:
stem_sentences["Sentence"]=stem_sentences["Sentence"].apply(lambda text: clean_frequent_expression(text))

In [106]:
stem_sentences["Sentence"][46]

'  üniversite bilgisayar , elektronik , matematik mühendis , '

In [107]:
stem_sentences["Sentence"]=stem_sentences["Sentence"].apply(lambda text : re.sub('i', 'i', text))

# CREATE NEW SKILLS DF WITH SPLIT (",")/ STEP 9

In [108]:
sentences_split = pd.DataFrame(columns=['PostId','SentenceId' ,'Skill'])

In [109]:
stem_sentences["Sentence"]=stem_sentences["Sentence"].apply(lambda text: re.split(",",str(text)))

In [110]:
stem_sentences["Sentence"][3]

['  erp pojelerinde test uzman ol ']

In [111]:
stem_sentences=stem_sentences.reset_index()

In [112]:
stem_sentences = stem_sentences.drop("index",axis=1)

In [113]:
stem_sentences

,PostId,Sentence,Language
0,Post0,"[ dünya ülke , fark dil yaklaşık ]",Türkçe
1,Post0,[ firma kullan erp yazılım SPCL38SPCL busines...,Türkçe
2,Post0,[ üniversite lisans ],Türkçe
3,Post0,[ erp pojelerinde test uzman ol ],Türkçe
4,Post0,[ analitik matematik düşün yön kuvvet ],Türkçe
...,...,...,...
14266,Post1119,"[ üniversite lisans program biri mezun , ter...",Türkçe
14267,Post1119,"[ sözlü yazılı i̇ngilizce bilgi sahip , ]",Türkçe
14268,Post1119,"[ erkek aday asker tamamla , ]",Türkçe
14269,Post1119,"[ analitik düşün , sorumluluk üstlen , deta...",Türkçe


In [114]:
for i in range(len(stem_sentences)):
    for j in range (len(stem_sentences["Sentence"][i])):
        sentences_split.loc[len(sentences_split.index)] = [stem_sentences["PostId"][i], "Sentence"+str(i), stem_sentences["Sentence"][i][j]] 
sentences_split.head()  

,PostId,SentenceId,Skill
0,Post0,Sentence0,dünya ülke
1,Post0,Sentence0,fark dil yaklaşık
2,Post0,Sentence1,firma kullan erp yazılım SPCL38SPCL business...
3,Post0,Sentence2,üniversite lisans
4,Post0,Sentence3,erp pojelerinde test uzman ol


In [115]:
sentences_split.drop(sentences_split[sentences_split['Skill'] == " "].index, inplace = True)

In [116]:
sentences_split.drop(sentences_split[sentences_split['Skill'] == ""].index, inplace = True)
sentences_split.reset_index(inplace=True)

In [117]:
sentences_split.head(60)

,index,PostId,SentenceId,Skill
0,0,Post0,Sentence0,dünya ülke
1,1,Post0,Sentence0,fark dil yaklaşık
2,2,Post0,Sentence1,firma kullan erp yazılım SPCL38SPCL business...
3,3,Post0,Sentence2,üniversite lisans
4,4,Post0,Sentence3,erp pojelerinde test uzman ol
5,5,Post0,Sentence4,analitik matematik düşün yön kuvvet
6,6,Post0,Sentence5,detay odak çalış
7,7,Post0,Sentence6,raporla yap
8,8,Post0,Sentence7,ekip çalış uyum
9,9,Post0,Sentence8,problem çöz yetenek sahip


In [118]:
sentences_split.to_csv("yazilim_uzmani_sentences_v3_split.csv")

In [119]:
sentences_split.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27835 entries, 0 to 27834
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   index       27835 non-null  int64 
 1   PostId      27835 non-null  object
 2   SentenceId  27835 non-null  object
 3   Skill       27835 non-null  object
dtypes: int64(1), object(3)
memory usage: 870.0+ KB


In [120]:
sentences_split["Skill"].value_counts().head(60)

                                                           536
 css                                                       294
 SPCL24SPCL script                                         216
 jquery                                                    191
 SPCL20SPCL                                                188
  SPCL20SPCL                                               173
  üniversite bilgisayar mühendis                           121
  üniversite                                               119
  takım çalış yatkın                                       116
 json                                                      105
 xml                                                        97
 ajax                                                       96
  analitik düşün                                            89
  erkek aday asker tamamla                                  83
 SPCL36SPCL                                                 82
  ekip çalış yatkın                                    

# MASKING / STEP 10

## FOR SOFT SKILLS

### Soft Skill Detection

In [121]:
soft_skill_dictionary = pd.read_excel(os.path.abspath('..')+"/Data/sentence_dictionary.xlsx")
soft_skill_dictionary["Orjinal Cümle"]=soft_skill_dictionary["Orjinal Cümle"].apply(lambda text : re.sub('i', 'i', text))
soft_skill_dictionary.tail()

,Orjinal Cümle,Soft Skill
90,önlisans mezun,önlisans mezunu
91,aday ara özellik,-
92,takım birlikte hareket,takım çalışmasına yatkın
93,bebekolog,-
94,tercihen,-


In [122]:
soft_skill_dictionary=dict(soft_skill_dictionary.values)

In [123]:
soft_skill_dictionary

{'asker': 'askerlik engelinin bulunmaması',
 'lise mezun': 'lise mezunu',
 'üniversite mezun': 'üniversite mezunu',
 'ingilizce': 'ingilizce',
 'takım çalış': 'takım çalışmasına yatkın',
 'ms office': 'ms office',
 'seyahat engel ': 'seyahat engeli olmayan',
 'ekip çalış ': 'ekip çalışmasına yatkın',
 'esnek çalış ': 'esnek çalışma saatlerine uyum sağlayabilen',
 'aktif araç': 'aktif araç kullanan',
 'diksiyon ': 'diksiyonu düzgün',
 'vardiya çalış': 'vardiyalı çalışabilecek',
 'lisans mezun': 'lisans mezunu',
 'üniversite lisans': 'lisans mezunu',
 'analitik düşün': 'analitik düşünebilen',
 'i̇letişim beceri': 'İletişim becerisi',
 'yoğun iş tempo ': 'yoğun iş temposuna ayak uydurabilen',
 'yoğun çalış tempo ': 'yoğun iş temposuna ayak uydurabilen',
 'i̇nsan ilişki başarı': 'İletişim becerisi',
 'sorumluluk sahip': 'sorumluluk sahibi',
 'i̇kna kabiliyet': 'ikna kabiliyeti yüksek',
 'mühendis bölüm mezun': 'üniversite mezunu',
 'vardiya sistem çalış': 'vardiyalı çalışabilecek',
 'dış g

In [124]:
def soft_skill_detection(text):
    value = text;
    for k,v in soft_skill_dictionary.items():
        if(k in text):
            return v
    return value       

In [125]:
sentences_split["Skill"] = sentences_split["Skill"].apply(lambda text: soft_skill_detection(text))

In [126]:
sentences_split["Skill"].value_counts().head(40)

-                                      1100
yazılım geliştirme                      831
üniversite mezunu                       699
askerlik engelinin bulunmaması          619
                                        536
takım çalışmasına yatkın                397
analitik düşünebilen                    332
 css                                    294
ekip çalışmasına yatkın                 260
 SPCL24SPCL script                      216
 jquery                                 191
 SPCL20SPCL                             188
  SPCL20SPCL                            173
problem çözme yeteneğine sahip          159
sorumluluk sahibi                       141
 json                                   105
sonuç odaklı                             98
 xml                                     97
 ajax                                    96
 SPCL36SPCL                              82
 bootstrap                               76
 yazılım mühendis                        75
seyahat engeli olmayan          

### CREATE DICTIONARY

In [127]:
soft_skill_mask = {}
reversed_soft_skill_mask = {}
line_counter=1
for k,v in soft_skill_dictionary.items():
    soft_skill_mask[v] = "SFT"+str(line_counter)+"SFT"
    reversed_soft_skill_mask["SFT"+str(line_counter)+"SFT"] = v
    line_counter +=1

In [128]:
soft_skill_mask

{'askerlik engelinin bulunmaması': 'SFT1SFT',
 'lise mezunu': 'SFT79SFT',
 'üniversite mezunu': 'SFT30SFT',
 'ingilizce': 'SFT4SFT',
 'takım çalışmasına yatkın': 'SFT90SFT',
 'ms office': 'SFT78SFT',
 'seyahat engeli olmayan': 'SFT7SFT',
 'ekip çalışmasına yatkın': 'SFT8SFT',
 'esnek çalışma saatlerine uyum sağlayabilen': 'SFT9SFT',
 'aktif araç kullanan': 'SFT10SFT',
 'diksiyonu düzgün': 'SFT11SFT',
 'vardiyalı çalışabilecek': 'SFT62SFT',
 'lisans mezunu': 'SFT14SFT',
 'analitik düşünebilen': 'SFT15SFT',
 'İletişim becerisi': 'SFT85SFT',
 'yoğun iş temposuna ayak uydurabilen': 'SFT18SFT',
 'sorumluluk sahibi': 'SFT61SFT',
 'ikna kabiliyeti yüksek': 'SFT21SFT',
 'dış görünümüne özen gösteren': 'SFT45SFT',
 'b sınıf ehliyet': 'SFT26SFT',
 'planlama': 'SFT27SFT',
 'sigara kullanmayan': 'SFT31SFT',
 'müşteri memnuniyetini sağlayabilen': 'SFT33SFT',
 'teknik resim': 'SFT34SFT',
 'prezantabl': 'SFT51SFT',
 'müşteri odaklı': 'SFT53SFT',
 'güler yüzlü': 'SFT38SFT',
 '-': 'SFT92SFT',
 'raporla

### MASKING

In [129]:
for key,val in soft_skill_mask.items():
    sentences_split["Skill"] = sentences_split["Skill"].apply(lambda text: text.replace(key," "+val+" "))  

In [130]:
sentences_split["Skill"].value_counts().head(60)

 SFT92SFT                                 1100
 SFT58SFT                                  831
 SFT30SFT                                  699
 SFT1SFT                                   619
                                           536
 SFT90SFT                                  397
 SFT15SFT                                  332
 css                                       294
 SFT8SFT                                   260
 SPCL24SPCL script                         216
 jquery                                    191
 SPCL20SPCL                                188
  SPCL20SPCL                               173
 SFT55SFT                                  159
 SFT61SFT                                  141
 json                                      105
 SFT43SFT                                   98
 xml                                        97
 ajax                                       96
 SPCL36SPCL                                 82
 bootstrap                                  76
 yazılım mühe

In [131]:
sentences_split.to_csv("yazilim_uzmani_sentences_v3_split.csv")

# CREATE NEW SKILLS DF WITH SPLIT (" ")/ STEP 11

In [132]:
sentences_split_2 = pd.DataFrame(columns=['PostId','SentenceId' ,'Skill'])

In [133]:
sentences_split["Skill"]=sentences_split["Skill"].apply(lambda text: re.split(" ",str(text)))

In [134]:
sentences_split["Skill"]

0                                      [, , dünya, ülke, ]
1                                [, fark, dil, yaklaşık, ]
2        [, , firma, kullan, erp, yazılım, SPCL38SPCL, ...
3                                           [, SFT14SFT, ]
4                [, , erp, pojelerinde, test, uzman, ol, ]
                               ...                        
27830                                       [, SFT15SFT, ]
27831                             [, sorumluluk, üstlen, ]
27832                               [, detay, önem, ver, ]
27833                                        [, SFT8SFT, ]
27834          [, araştır, disiplin, çalış, konu, iddia, ]
Name: Skill, Length: 27835, dtype: object

In [135]:
sentences_split=sentences_split.reset_index()

In [136]:
sentences_split = sentences_split.drop(["level_0"],axis=1)

In [137]:
sentences_split

,index,PostId,SentenceId,Skill
0,0,Post0,Sentence0,"[, , dünya, ülke, ]"
1,1,Post0,Sentence0,"[, fark, dil, yaklaşık, ]"
2,2,Post0,Sentence1,"[, , firma, kullan, erp, yazılım, SPCL38SPCL, ..."
3,3,Post0,Sentence2,"[, SFT14SFT, ]"
4,4,Post0,Sentence3,"[, , erp, pojelerinde, test, uzman, ol, ]"
...,...,...,...,...
27830,34854,Post1119,Sentence14269,"[, SFT15SFT, ]"
27831,34855,Post1119,Sentence14269,"[, sorumluluk, üstlen, ]"
27832,34856,Post1119,Sentence14269,"[, detay, önem, ver, ]"
27833,34858,Post1119,Sentence14270,"[, SFT8SFT, ]"


In [138]:
for i in range(len(sentences_split)):
    for j in range (len(sentences_split["Skill"][i])):
        sentences_split_2.loc[len(sentences_split_2.index)] = [sentences_split["PostId"][i],sentences_split["SentenceId"][i], sentences_split["Skill"][i][j]] 
sentences_split_2.head()  

,PostId,SentenceId,Skill
0,Post0,Sentence0,
1,Post0,Sentence0,
2,Post0,Sentence0,dünya
3,Post0,Sentence0,ülke
4,Post0,Sentence0,


In [139]:
sentences_split_2.tail()

,PostId,SentenceId,Skill
168520,Post1119,Sentence14270,disiplin
168521,Post1119,Sentence14270,çalış
168522,Post1119,Sentence14270,konu
168523,Post1119,Sentence14270,iddia
168524,Post1119,Sentence14270,


In [140]:
sentences_split_2.drop(sentences_split_2[sentences_split_2['Skill'] == " "].index, inplace = True)

In [141]:
sentences_split_2.drop(sentences_split_2[sentences_split_2['Skill'] == ""].index, inplace = True)
sentences_split_2.reset_index(inplace=True)

In [142]:
sentences_split_2.head(60)

,index,PostId,SentenceId,Skill
0,2,Post0,Sentence0,dünya
1,3,Post0,Sentence0,ülke
2,6,Post0,Sentence0,fark
3,7,Post0,Sentence0,dil
4,8,Post0,Sentence0,yaklaşık
5,12,Post0,Sentence1,firma
6,13,Post0,Sentence1,kullan
7,14,Post0,Sentence1,erp
8,15,Post0,Sentence1,yazılım
9,16,Post0,Sentence1,SPCL38SPCL


In [143]:
sentences_split_2.to_csv("yazilim_uzmani_sentences_v4_split.csv")

In [144]:
sentences_split_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102344 entries, 0 to 102343
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   index       102344 non-null  int64 
 1   PostId      102344 non-null  object
 2   SentenceId  102344 non-null  object
 3   Skill       102344 non-null  object
dtypes: int64(1), object(3)
memory usage: 3.1+ MB


In [145]:
sentences_split_2["Skill"].value_counts().head(60)

konu          2307
sahip         2158
geliş         2072
bilgi         1893
yazılım       1498
uygula        1355
ol            1322
proje         1229
web           1185
deneyim       1141
SFT92SFT      1126
tecrübe       1109
teknoloji     1092
çalış         1050
SPCL24SPCL     867
SFT58SFT       831
hakim          743
kullan         730
SFT30SFT       699
yap            678
al             623
SFT1SFT        619
tasarım        598
bil            589
SPCL20SPCL     577
yeni           572
sistem         548
script         545
et             542
görev          531
css            515
i̇ngilizce     510
uzman          485
sql            477
servis         467
ara            445
programla      429
takip          427
mühendis       414
SPCL70SPCL     412
SPCL36SPCL     408
derece         405
ver            402
süreç          399
SFT90SFT       397
dil            396
SPCL45SPCL     392
test           387
jquery         386
kod            377
taban          370
hak            369
çözüm       

# CLEAN VERBS / STEP 12

Kökü fiil olan kelimelerin temizlenmesi

In [146]:
sentences_split = pd.read_csv("yazilim_uzmani_sentences_v4_split.csv")
sentences_split = sentences_split.drop(["Unnamed: 0","index"],axis=1)

In [147]:
sentences_split.drop(sentences_split[sentences_split['Skill'] == " "].index, inplace = True)

In [148]:
sentences_split.drop(sentences_split[sentences_split['Skill'] == ""].index, inplace = True)
sentences_split.reset_index(inplace=True)
sentences_split=sentences_split.drop("index",axis=1)
sentences_split

,PostId,SentenceId,Skill
0,Post0,Sentence0,dünya
1,Post0,Sentence0,ülke
2,Post0,Sentence0,fark
3,Post0,Sentence0,dil
4,Post0,Sentence0,yaklaşık
...,...,...,...
102339,Post1119,Sentence14270,araştır
102340,Post1119,Sentence14270,disiplin
102341,Post1119,Sentence14270,çalış
102342,Post1119,Sentence14270,konu


In [149]:
sentences_split["Skill"]=sentences_split["Skill"].apply(lambda text : re.sub('i̇', 'i', text))

In [150]:
def clean_verbs(text):
    stem_text=text
    analysis: jpype.java.util.ArrayList = (
    morphology.analyzeAndDisambiguate(text).bestAnalysis()
    )
    for i in analysis:
        if ("UNK" in  str(i.getLemmas()[0])) :
            stem_text=str(i.getStem())
        else:
            stem_text = " "
    return stem_text

In [151]:
clean_verbs("i̇yi")

'i̇yi'

In [152]:
sentences_split["Skill"]=sentences_split["Skill"].apply(lambda text: clean_verbs(str(text)))     

In [153]:
sentences_split["Skill"]

0          
1          
2          
3          
4          
         ..
102339     
102340     
102341     
102342     
102343     
Name: Skill, Length: 102344, dtype: object

In [154]:
sentences_split.drop(sentences_split[sentences_split['Skill'] == " "].index, inplace = True)
sentences_split.drop(sentences_split[sentences_split['Skill'] == ""].index, inplace = True)
sentences_split.drop(sentences_split[sentences_split['Skill'] == "'"].index, inplace = True)
sentences_split.drop(sentences_split[sentences_split['Skill'] == "i"].index, inplace = True)
sentences_split.drop(sentences_split[sentences_split['Skill'] == "ld"].index, inplace = True)
sentences_split.drop(sentences_split[sentences_split['Skill'] == "yr"].index, inplace = True)

In [155]:
sentences_split

,PostId,SentenceId,Skill
7,Post0,Sentence1,erp
9,Post0,Sentence1,SPCL38SPCL
12,Post0,Sentence1,SPCL29SPCL
26,Post0,Sentence2,SFT14SFT
27,Post0,Sentence3,erp
...,...,...,...
102324,Post1119,Sentence14266,SFT14SFT
102325,Post1119,Sentence14266,SFT92SFT
102331,Post1119,Sentence14268,SFT1SFT
102332,Post1119,Sentence14269,SFT15SFT


In [156]:
sentences_split["Skill"].value_counts().to_csv("yazilim_uzmani_skills.csv")

### CLEAN IRRELEVANT THINGS

In [157]:
sentences_split["Skill"]=sentences_split["Skill"].replace(to_replace=["'|§|\||>|²|¦"],value=" ",regex=True,inplace=False)
sentences_split["Skill"]=sentences_split["Skill"].str.replace(r'\\'," ")

# REVERSED MASKING / STEP 13

Maskelenen tüm skillerin geri alınması

### SPECIAL SKILLS

In [158]:
special_terms_dictionary = {}
reversed_special_terms_dictionary = {}
with open(os.path.abspath('..')+"/Data/_special_terms.txt",encoding="utf8") as file:
    line_counter=1
    for line in file:
        line=line.lower()
        special_terms_dictionary[line.strip("\n")] = "SPCL"+str(line_counter)+"SPCL"
        reversed_special_terms_dictionary["SPCL"+str(line_counter)+"SPCL"] = line.strip()
        line_counter +=1

In [159]:
for key,val in reversed_special_terms_dictionary.items():
    sentences_split["Skill"] = sentences_split["Skill"].apply(lambda text: text.replace(key,val))    

### SOFT SKILLS

In [160]:
soft_skill_mask = {}
reversed_soft_skill_mask = {}
line_counter=1
for k,v in soft_skill_dictionary.items():
    soft_skill_mask[v] = "SFT"+str(line_counter)+"SFT"
    reversed_soft_skill_mask["SFT"+str(line_counter)+"SFT"] = v
    line_counter +=1

In [161]:
for key,val in reversed_soft_skill_mask.items():
    sentences_split["Skill"] = sentences_split["Skill"].apply(lambda text: text.replace(key,val))    

In [162]:
sentences_split["Skill"]=sentences_split["Skill"].replace(to_replace=["-"],value=" ",regex=True,inplace=False)
sentences_split.drop(sentences_split[sentences_split['Skill'] == " "].index, inplace = True)

In [163]:
sentences_split["Skill"].value_counts().head(60)

java                                   867
yazılım geliştirme                     831
üniversite mezunu                      699
askerlik engelinin bulunmaması         619
html                                   577
css                                    515
sql                                    477
c#                                     412
mvc                                    408
takım çalışmasına yatkın               397
.net                                   392
jquery                                 386
asp.net                                377
analitik düşünebilen                   332
api                                    302
ekip çalışmasına yatkın                260
go                                     257
php                                    234
json                                   217
xml                                    212
soap                                   207
problem çözme yeteneğine sahip         159
bootstrap                              157
entity     

# REPLACE FOR "YAZILIM UZMANI" SKILLS

Sadece yazılım uzmanı için yapılacak olran ara işlemdir. Diğer pozisyonlar için bu işlemin yapılmasına gerek yok.

In [164]:
sentences_split["Skill"]=sentences_split["Skill"].str.replace("restfull","restful")
sentences_split["Skill"]=sentences_split["Skill"].str.replace("object\soriented","oop",regex=True)
sentences_split["Skill"]=sentences_split["Skill"].str.replace("(?:^|\W)object(?:$|\W)","oop",regex=True)
sentences_split["Skill"]=sentences_split["Skill"].str.replace("asp.*","asp.net",regex=True)
sentences_split["Skill"]=sentences_split["Skill"].str.replace("uuml","uml")
sentences_split["Skill"]=sentences_split["Skill"].str.replace("önyüz","frontend")
sentences_split["Skill"]=sentences_split["Skill"].str.replace("ritma.*","ritma",regex=True)
sentences_split["Skill"]=sentences_split["Skill"].str.replace("bootsrap","bootstrap")
sentences_split["Skill"]=sentences_split["Skill"].str.replace("plsql","pg/sql")
sentences_split["Skill"]=sentences_split["Skill"].str.replace("apis","api")
sentences_split["Skill"]=sentences_split["Skill"].str.replace("mikroservis","microservices")
sentences_split["Skill"]=sentences_split["Skill"].str.replace("nodejs","node.js")
sentences_split["Skill"]=sentences_split["Skill"].str.replace("entity.*","entityframework",regex=True)
sentences_split["Skill"]=sentences_split["Skill"].str.replace("fullstack","full stack")
sentences_split["Skill"]=sentences_split["Skill"].str.replace("backend","back end")
sentences_split["Skill"]=sentences_split["Skill"].str.replace("frontend","front end")

# CLEANING LOW FREQUENCY SKIILS / STEP 14

In [165]:
sentences_split=sentences_split.groupby('Skill').filter(lambda x : len(x)>11)
sentences_split.reset_index(inplace=True)

In [166]:
sentences_split

,index,PostId,SentenceId,Skill
0,7,Post0,Sentence1,erp
1,9,Post0,Sentence1,sap
2,12,Post0,Sentence1,go
3,26,Post0,Sentence2,lisans mezunu
4,27,Post0,Sentence3,erp
...,...,...,...,...
16334,102323,Post1119,Sentence14265,aktif araç kullanan
16335,102324,Post1119,Sentence14266,lisans mezunu
16336,102331,Post1119,Sentence14268,askerlik engelinin bulunmaması
16337,102332,Post1119,Sentence14269,analitik düşünebilen


In [167]:
sentences_split=sentences_split.drop_duplicates()

In [168]:
sentences_split.to_csv("yazilim_uzmani_skills_v2.csv")
sentences_split["Skill"].value_counts().to_csv("yazilim_uzmani_skills_counts_v1.csv")